In [96]:
# import relevant packages
import pandas as pd, numpy as np
from sklearn import model_selection, linear_model
from unidecode import unidecode

In [97]:
# read in the data
election_results_df = pd.read_csv('countypres_2000-2020.csv')
election_results_df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [98]:
# read in the data
election_results_df = pd.read_csv('countypres_2000-2020.csv')
demographics_df = pd.read_csv('cc-est2022-all.csv', encoding = 'latin-1')
election_results_df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [99]:
demographics_df.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,58802,28761,30041,...,854,734,112,97,40,30,22,24,19,11
1,50,1,1,Alabama,Autauga County,1,1,3443,1790,1653,...,83,54,14,9,3,1,9,1,3,1
2,50,1,1,Alabama,Autauga County,1,2,3613,1849,1764,...,81,68,5,8,7,3,0,1,1,2
3,50,1,1,Alabama,Autauga County,1,3,4131,2121,2010,...,81,81,10,10,1,3,2,2,2,1
4,50,1,1,Alabama,Autauga County,1,4,3963,2003,1960,...,72,70,8,9,5,2,2,4,3,2


**Data preprocessing and cleaning**

First, we need to join our two datasets, our demographic dataset of predictors and our election results dataset, on county, but we will need to do a bit of cleaning.

In [100]:
# cleaning series to join dataframes
demographics_df['county'] = demographics_df['CTYNAME'].apply(lambda x: x.replace(" County", "")) + demographics_df['STNAME']

In [101]:
demographics_df['county'] = demographics_df['county'].str.replace(" " , "").str.lower()
election_results_df['county_name'] = election_results_df['county_name'].str.replace(" " , "").str.lower()
election_results_df['state'] = election_results_df['state'].str.replace(" " , "").str.lower()
election_results_df['county'] = election_results_df['county_name'] + election_results_df['state']

In [102]:
# filtering for 2020 election only, where there is overlap between the two datasets; note this will exclude Virginia's 38 independent cities
election_results_df = election_results_df[(election_results_df['year'] == 2020)&(election_results_df['mode'] == "TOTAL")]

In [103]:
# filtering for year = 2020 and all age groups
demographics_df = demographics_df[(demographics_df['YEAR'] == 1)&(demographics_df['AGEGRP'] == 0)]

In [104]:
# merge datasets
data = election_results_df.merge(demographics_df, how = 'left', on = 'county')

In [105]:
data.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,2020,alabama,AL,autauga,1001.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,7503,27770,...,854.0,734.0,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0
1,2020,alabama,AL,autauga,1001.0,US PRESIDENT,OTHER,OTHER,429,27770,...,854.0,734.0,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0
2,2020,alabama,AL,autauga,1001.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,19838,27770,...,854.0,734.0,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0
3,2020,alabama,AL,baldwin,1003.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,24578,109679,...,5256.0,4806.0,327.0,334.0,305.0,241.0,101.0,95.0,95.0,73.0
4,2020,alabama,AL,baldwin,1003.0,US PRESIDENT,OTHER,OTHER,1557,109679,...,5256.0,4806.0,327.0,334.0,305.0,241.0,101.0,95.0,95.0,73.0


In [106]:
with pd.option_context('display.max_rows', None,):
    print(data[data.isnull().any(axis=1)].to_string())

      year               state state_po         county_name  county_fips        office          candidate        party  candidatevotes  totalvotes   version   mode                                county  SUMLEV  STATE  COUNTY                STNAME               CTYNAME  YEAR  AGEGRP   TOT_POP  TOT_MALE  TOT_FEMALE   WA_MALE  WA_FEMALE   BA_MALE  BA_FEMALE  IA_MALE  IA_FEMALE  AA_MALE  AA_FEMALE  NA_MALE  NA_FEMALE  TOM_MALE  TOM_FEMALE  WAC_MALE  WAC_FEMALE  BAC_MALE  BAC_FEMALE  IAC_MALE  IAC_FEMALE  AAC_MALE  AAC_FEMALE  NAC_MALE  NAC_FEMALE   NH_MALE  NH_FEMALE  NHWA_MALE  NHWA_FEMALE  NHBA_MALE  NHBA_FEMALE  NHIA_MALE  NHIA_FEMALE  NHAA_MALE  NHAA_FEMALE  NHNA_MALE  NHNA_FEMALE  NHTOM_MALE  NHTOM_FEMALE  NHWAC_MALE  NHWAC_FEMALE  NHBAC_MALE  NHBAC_FEMALE  NHIAC_MALE  NHIAC_FEMALE  NHAAC_MALE  NHAAC_FEMALE  NHNAC_MALE  NHNAC_FEMALE   H_MALE  H_FEMALE  HWA_MALE  HWA_FEMALE  HBA_MALE  HBA_FEMALE  HIA_MALE  HIA_FEMALE  HAA_MALE  HAA_FEMALE  HNA_MALE  HNA_FEMALE  HTOM_MALE  HTOM_FEMALE  

Above we can observe the counties that did not have a demographic match. There are a few separate cases which are explained below: 

1. Alaska: This is broken down into Districts/boroughs by the election data, but it is broken into territories for the demographic data, and the two are not comparable. Hence, Alaska is excluded from this analysis. 
2. Connecticut: Similarly for CT, regions are split into "capitol planning areas" for demographic data and thus uncomparable. Connecticut actually doesn't have county-level governments to collect county-level voting data. 
3. D.C.: Just the FIPS code is missing, which makes sense because D.C. does not have a FIPS code. 
4. Lousiana: This state calls its counties "parishes", so the initial data cleaning was not sufficient. We can slice off "parish" similarly. 
5. Minnesota: demographics dataset spells "St. Louis" while election results dataset spells "Saint Louis", which indicates a data reading error
6. Missouri: St. Louis City and St. Louis County both exist, and this likely caused issues with a merge. Kansas City is also included when it is not a county, so we will exclude it. St. Louis City is an independent city and has no "county" per se. 
7. Nevada: Carson City is an independent city as of 1969 and thus had data reading errors as it did not end in "county"
8. New Mexico: Demographics dataset includes ñ while the elections dataset does not
9. Rhode Island: It's unclear after further investigation what is the federal precinct, as there is no mention online. Bucketed with Alaska and Connecticut as unusable.

Below, we clean the data.

In [107]:
# fix New Mexico
demographics_df['county'] = demographics_df['county'].apply(lambda x: unidecode(x))

In [108]:
# handling Louisiana exception
demographics_df['county'] = demographics_df['county'].str.replace("parish", "")

In [109]:
# replace "Saint" with St. to fix Minnesota/Missouri issue
election_results_df['county'] = election_results_df['county'].str.replace("saint", "st.")

In [124]:
demographics_df

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,county
0,50,1,1,Alabama,Autauga County,1,0,58802,28761,30041,...,734,112,97,40,30,22,24,19,11,autaugaalabama
76,50,1,3,Alabama,Baldwin County,1,0,231761,113388,118373,...,4806,327,334,305,241,101,95,95,73,baldwinalabama
152,50,1,5,Alabama,Barbour County,1,0,25224,13325,11899,...,431,77,64,73,36,11,7,22,16,barbouralabama
228,50,1,7,Alabama,Bibb County,1,0,22300,11888,10412,...,257,36,20,13,13,7,6,20,4,bibbalabama
304,50,1,9,Alabama,Blount County,1,0,59130,29451,29679,...,2502,87,73,79,72,13,17,32,23,blountalabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238564,50,56,37,Wyoming,Sweetwater County,1,0,42267,21998,20269,...,2991,91,75,217,190,28,37,17,11,sweetwaterwyoming
238640,50,56,39,Wyoming,Teton County,1,0,23323,12202,11121,...,1555,29,19,109,80,15,12,13,11,tetonwyoming
238716,50,56,41,Wyoming,Uinta County,1,0,20446,10430,10016,...,889,25,22,97,126,7,12,7,5,uintawyoming
238792,50,56,43,Wyoming,Washakie County,1,0,7682,3945,3737,...,472,12,12,53,53,5,6,4,2,washakiewyoming


In [111]:
# merge datasets
cleaned_data = election_results_df.merge(demographics_df, how = 'left', on = 'county')

In [112]:
with pd.option_context('display.max_rows', None,):
    print(cleaned_data[cleaned_data.isnull().any(axis=1)].to_string())

      year               state state_po         county_name  county_fips        office          candidate        party  candidatevotes  totalvotes   version   mode                                county  SUMLEV  STATE  COUNTY                STNAME               CTYNAME  YEAR  AGEGRP   TOT_POP  TOT_MALE  TOT_FEMALE   WA_MALE  WA_FEMALE   BA_MALE  BA_FEMALE  IA_MALE  IA_FEMALE  AA_MALE  AA_FEMALE  NA_MALE  NA_FEMALE  TOM_MALE  TOM_FEMALE  WAC_MALE  WAC_FEMALE  BAC_MALE  BAC_FEMALE  IAC_MALE  IAC_FEMALE  AAC_MALE  AAC_FEMALE  NAC_MALE  NAC_FEMALE   NH_MALE  NH_FEMALE  NHWA_MALE  NHWA_FEMALE  NHBA_MALE  NHBA_FEMALE  NHIA_MALE  NHIA_FEMALE  NHAA_MALE  NHAA_FEMALE  NHNA_MALE  NHNA_FEMALE  NHTOM_MALE  NHTOM_FEMALE  NHWAC_MALE  NHWAC_FEMALE  NHBAC_MALE  NHBAC_FEMALE  NHIAC_MALE  NHIAC_FEMALE  NHAAC_MALE  NHAAC_FEMALE  NHNAC_MALE  NHNAC_FEMALE   H_MALE  H_FEMALE  HWA_MALE  HWA_FEMALE  HBA_MALE  HBA_FEMALE  HIA_MALE  HIA_FEMALE  HAA_MALE  HAA_FEMALE  HNA_MALE  HNA_FEMALE  HTOM_MALE  HTOM_FEMALE  

In [113]:
cleaned_data = cleaned_data.dropna().drop(columns = ['year', 'state', 'YEAR', 'AGEGRP','state_po', 'county_fips', 'office', 'version','mode', 'county_name', 'SUMLEV', 'STNAME', 'CTYNAME', 'STATE', 'COUNTY'])

In [114]:
cleaned_data

,candidate,party,candidatevotes,totalvotes,county,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,JOSEPH R BIDEN JR,DEMOCRAT,7503,27770,autaugaalabama,58802.0,28761.0,30041.0,22036.0,22601.0,...,854.0,734.0,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0
1,OTHER,OTHER,429,27770,autaugaalabama,58802.0,28761.0,30041.0,22036.0,22601.0,...,854.0,734.0,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0
2,DONALD J TRUMP,REPUBLICAN,19838,27770,autaugaalabama,58802.0,28761.0,30041.0,22036.0,22601.0,...,854.0,734.0,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0
3,JOSEPH R BIDEN JR,DEMOCRAT,24578,109679,baldwinalabama,231761.0,113388.0,118373.0,99054.0,103205.0,...,5256.0,4806.0,327.0,334.0,305.0,241.0,101.0,95.0,95.0,73.0
4,OTHER,OTHER,1557,109679,baldwinalabama,231761.0,113388.0,118373.0,99054.0,103205.0,...,5256.0,4806.0,327.0,334.0,305.0,241.0,101.0,95.0,95.0,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10054,DONALD J TRUMP,REPUBLICAN,3245,4032,washakiewyoming,7682.0,3945.0,3737.0,3750.0,3536.0,...,518.0,472.0,12.0,12.0,53.0,53.0,5.0,6.0,4.0,2.0
10055,JOSEPH R BIDEN JR,DEMOCRAT,360,3560,westonwyoming,6840.0,3724.0,3116.0,3476.0,2897.0,...,157.0,112.0,5.0,6.0,25.0,27.0,3.0,0.0,2.0,1.0
10056,JO JORGENSEN,LIBERTARIAN,46,3560,westonwyoming,6840.0,3724.0,3116.0,3476.0,2897.0,...,157.0,112.0,5.0,6.0,25.0,27.0,3.0,0.0,2.0,1.0
10057,OTHER,OTHER,47,3560,westonwyoming,6840.0,3724.0,3116.0,3476.0,2897.0,...,157.0,112.0,5.0,6.0,25.0,27.0,3.0,0.0,2.0,1.0


In [115]:
# creating a column for candidate vote %
cleaned_data['pct_vote'] = cleaned_data['candidatevotes'] / cleaned_data['totalvotes']

In [116]:
# consolidating party with candidate
cleaned_data['Candidate'] = cleaned_data['candidate'] + " - " + cleaned_data['party']
cleaned_data = cleaned_data.drop(columns=['candidate', 'party'])

In [117]:
cleaned_data.index = cleaned_data['county']
cleaned_data

,candidatevotes,totalvotes,county,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,...,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,pct_vote,Candidate
county,,,,,,,,,,,,,,,,,,,,,
autaugaalabama,7503,27770,autaugaalabama,58802.0,28761.0,30041.0,22036.0,22601.0,5684.0,6339.0,...,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0,0.270184,JOSEPH R BIDEN JR - DEMOCRAT
autaugaalabama,429,27770,autaugaalabama,58802.0,28761.0,30041.0,22036.0,22601.0,5684.0,6339.0,...,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0,0.015448,OTHER - OTHER
autaugaalabama,19838,27770,autaugaalabama,58802.0,28761.0,30041.0,22036.0,22601.0,5684.0,6339.0,...,112.0,97.0,40.0,30.0,22.0,24.0,19.0,11.0,0.714368,DONALD J TRUMP - REPUBLICAN
baldwinalabama,24578,109679,baldwinalabama,231761.0,113388.0,118373.0,99054.0,103205.0,9969.0,10288.0,...,327.0,334.0,305.0,241.0,101.0,95.0,95.0,73.0,0.224090,JOSEPH R BIDEN JR - DEMOCRAT
baldwinalabama,1557,109679,baldwinalabama,231761.0,113388.0,118373.0,99054.0,103205.0,9969.0,10288.0,...,327.0,334.0,305.0,241.0,101.0,95.0,95.0,73.0,0.014196,OTHER - OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
washakiewyoming,3245,4032,washakiewyoming,7682.0,3945.0,3737.0,3750.0,3536.0,24.0,15.0,...,12.0,12.0,53.0,53.0,5.0,6.0,4.0,2.0,0.804812,DONALD J TRUMP - REPUBLICAN
westonwyoming,360,3560,westonwyoming,6840.0,3724.0,3116.0,3476.0,2897.0,40.0,14.0,...,5.0,6.0,25.0,27.0,3.0,0.0,2.0,1.0,0.101124,JOSEPH R BIDEN JR - DEMOCRAT
westonwyoming,46,3560,westonwyoming,6840.0,3724.0,3116.0,3476.0,2897.0,40.0,14.0,...,5.0,6.0,25.0,27.0,3.0,0.0,2.0,1.0,0.012921,JO JORGENSEN - LIBERTARIAN


In [118]:
X = cleaned_data.drop(columns=['county', 'candidatevotes', 'totalvotes', 'Candidate', 'pct_vote'])
y = cleaned_data['pct_vote']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=0)

In [119]:
lr = linear_model.LinearRegression()
reg = lr.fit(X_train, y_train)
print(f"R^2: {reg.score(X_train, y_train):.2%}")

R^2: 0.21%


In [120]:
lasso = linear_model.Lasso(alpha=0.1, max_iter=10000)
l_reg = lasso.fit(X_train, y_train)
print(f"R^2: {l_reg.score(X_train, y_train):.2%}")

R^2: 0.09%


C:\Users\57185\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.943e+02, tolerance: 6.423e-02
  model = cd_fast.enet_coordinate_descent(


The linear model appears to be slightly better. However, we observe low predictability on the county-level as to the percent victory each candidate will achieve based on demographic and gender splits. This provides some evidence that "identity politics" may not actually be as true as many believe, given that these featuers are poor predictors for the percentage the candidate wins.

It should be noted that this analysis was in 2020, and that this study does not control for time. Future analyses could investigate this relationship across multiple presidential elections or even senate/house elections. In addition, depending on the data availability, we could attempt to run this analysis with another country.